monitor RW {

    int read_count = 0
    boolean is_writing = false

    condition can_read
    condition can_write

    start_read():
        while is_writing:
            wait(can_read)
        read_count++

    end_read():
        read_count--
        if read_count == 0:
            signal(can_write)

    start_write():
        while read_count > 0 or is_writing:
            wait(can_write)
        is_writing = true

    end_write():
        is_writing = false
        broadcast(can_read)
}

In [2]:
import threading
import time
import random

# =========================================================
# Monitor: Single Writer - Multiple Readers
# =========================================================
class ReaderWriterMonitor:
    def __init__(self):
        self.lock = threading.Lock()
        self.can_read = threading.Condition(self.lock)
        self.can_write = threading.Condition(self.lock)

        self.read_count = 0
        self.is_writing = False

    def start_read(self, reader_id):
        with self.lock:
            while self.is_writing:
                self.can_read.wait()
            self.read_count += 1
            print(f"[Reader {reader_id}] started reading (readers={self.read_count})", flush=True)

    def end_read(self, reader_id):
        with self.lock:
            self.read_count -= 1
            print(f"[Reader {reader_id}] finished reading (readers={self.read_count})", flush=True)
            if self.read_count == 0:
                self.can_write.notify()

    def start_write(self):
        with self.lock:
            while self.read_count > 0 or self.is_writing:
                self.can_write.wait()
            self.is_writing = True
            print("[Writer] started writing", flush=True)

    def end_write(self):
        with self.lock:
            self.is_writing = False
            print("[Writer] finished writing", flush=True)
            self.can_read.notify_all()


# =========================================================
# Reader and Writer threads
# =========================================================
def reader(monitor, reader_id):
    for _ in range(3):
        time.sleep(random.uniform(0.1, 1))
        monitor.start_read(reader_id)
        time.sleep(1)
        monitor.end_read(reader_id)

def writer(monitor):
    for _ in range(3):
        time.sleep(2)
        monitor.start_write()
        time.sleep(2)
        monitor.end_write()


# =========================================================
# Main
# =========================================================
monitor = ReaderWriterMonitor()

readers = [
    threading.Thread(target=reader, args=(monitor, i))
    for i in range(3)
]

writer_thread = threading.Thread(target=writer, args=(monitor,))

for r in readers:
    r.start()

writer_thread.start()

for r in readers:
    r.join()

writer_thread.join()

print("Execution complete.")

[Reader 2] started reading (readers=1)
[Reader 0] started reading (readers=2)
[Reader 1] started reading (readers=3)
[Reader 2] finished reading (readers=2)
[Reader 0] finished reading (readers=1)
[Reader 1] finished reading (readers=0)
[Reader 0] started reading (readers=1)
[Reader 2] started reading (readers=2)
[Reader 1] started reading (readers=3)
[Reader 0] finished reading (readers=2)
[Reader 0] started reading (readers=3)
[Reader 2] finished reading (readers=2)
[Reader 1] finished reading (readers=1)
[Reader 0] finished reading (readers=0)
[Writer] started writing
[Writer] finished writing
[Reader 1] started reading (readers=1)
[Reader 2] started reading (readers=2)
[Reader 1] finished reading (readers=1)
[Reader 2] finished reading (readers=0)
[Writer] started writing
[Writer] finished writing
[Writer] started writing
[Writer] finished writing
Execution complete.
